## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-11-13-05-12 +0000,bbc,Panic in France as children fall victim to let...,https://www.bbc.com/news/articles/cr7l0nyxxygo...
1,2025-12-11-13-03-53 +0000,wapo,"Every year, she thanks the trooper for the arr...",https://www.washingtonpost.com/lifestyle/2025/...
2,2025-12-11-13-00-00 +0000,missionlocal,Why is San Francisco’s pro-housing mayor oppos...,https://missionlocal.org/2025/12/sf-marina-saf...
3,2025-12-11-12-56-41 +0000,bbc,Jamie Oliver on the return of his restaurant c...,https://www.bbc.com/news/videos/cn8evm3d8r3o?a...
4,2025-12-11-12-56-03 +0000,wapo,Nobel laureate calls Venezuela a ‘criminal hub...,https://www.washingtonpost.com/world/2025/12/1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2453/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,49
25,new,24
565,fed,23
35,venezuela,16
199,off,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
132,2025-12-11-04-00-00 +0000,wsj,President Trump wants lower interest rates and...,https://www.wsj.com/economy/central-banking/fe...,148
181,2025-12-10-23-56-02 +0000,nypost,Trump says he wants a new owner for CNN as par...,https://nypost.com/2025/12/10/media/trump-says...,116
168,2025-12-11-00-32-00 +0000,wsj,Fed Chair Jerome Powell said that Fed staffers...,https://www.wsj.com/economy/jobs/fed-chair-jer...,106
128,2025-12-11-04-17-57 +0000,wapo,U.S. seizes ‘very large’ oil tanker off Venezu...,https://www.washingtonpost.com/world/2025/12/1...,103
108,2025-12-11-06-52-47 +0000,nyt,Trump Says U.S. Seized Oil Tanker Off Venezuel...,https://www.nytimes.com/2025/12/10/us/politics...,98


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
132,148,2025-12-11-04-00-00 +0000,wsj,President Trump wants lower interest rates and...,https://www.wsj.com/economy/central-banking/fe...
213,77,2025-12-10-22-35-53 +0000,bbc,Watch: Video shows US military seizing oil tan...,https://www.bbc.com/news/videos/c9vj1jnmekko?a...
274,58,2025-12-10-20-09-00 +0000,wsj,The Journal answered subscribers’ questions ab...,https://www.wsj.com/economy/central-banking/li...
336,55,2025-12-10-17-09-40 +0000,nypost,Machado’s daughter accepts Nobel Peace Prize i...,https://nypost.com/2025/12/10/world-news/macha...
41,53,2025-12-11-11-03-10 +0000,nypost,Louisiana Dem Rep. Cleo Fields quietly bought ...,https://nypost.com/2025/12/11/us-news/rep-cleo...
212,51,2025-12-10-22-37-00 +0000,wsj,Citizens of countries including the U.K. and F...,https://www.wsj.com/politics/policy/trump-admi...
369,47,2025-12-10-15-11-40 +0000,nypost,Husband charged with murdering Miss Switzerlan...,https://nypost.com/2025/12/10/world-news/husba...
168,46,2025-12-11-00-32-00 +0000,wsj,Fed Chair Jerome Powell said that Fed staffers...,https://www.wsj.com/economy/jobs/fed-chair-jer...
192,44,2025-12-10-23-21-00 +0000,wsj,More Republicans in the House and Senate are b...,https://www.wsj.com/politics/policy/some-repub...
270,41,2025-12-10-20-21-00 +0000,wsj,Judge Orders Trump Administration to End Natio...,https://www.wsj.com/us-news/law/judge-orders-t...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
